# Usermore soap case study

Versão 1.1 Guilherme Fernandes 15:55 10/maio/2025 

In [1]:
from pprint import pprint

from gurobipy import Model, GRB

## 1. Defining the sets

In [2]:
plants = [
    "Covington",
    "NewYork",
    "Arlington",
    "LongBeach",
]

existing_warehouses = [
    "Atlanta",
    "Boston",
    "Buffalo",
    "Chicago",
    "Cleveland",
    "Davenport",
    "Detroit",
    "GrandRapids",
    "Greensboro",
    "KansasCity",
    "Baltimore",
    "Memphis",
    "Milwaukee",
    "Orlando",
    "Pittsburgh",
    "Portland",
    "WestSac",
    "WestChester",
]

potential_warehouses = [
    "Albuquerque",
    "Billings",
    "Denver",
    "ElPaso",
    "CampHill",
    "Houston",
    "LasVegas",
    "Minneapolis",
    "NewOrleans",
    "Phoenix",
    "Richmond",
    "StLouis",
    "SaltLakeCity",
    "SanAntonio",
    "Seattle",
    "Spokane",
    "SanFrancisco",
    "Indianapolis",
    "Louisville",
    "Columbus",
    "NewYork",
    "Hartford",
    "Miami",
    "Mobile",
    "Memphis2",
    "Chicago2",
]

# Merge existing + potential
warehouses = existing_warehouses + potential_warehouses

print("Number of plants: ", len(plants))
print("Number of existing warehouses: ", len(existing_warehouses))
print("Number of potential warehouses: ", len(potential_warehouses))
print("Number of total warehouses: ", len(warehouses))
print("Obs.: Each plant is also an existing warehouse")

Number of plants:  4
Number of existing warehouses:  18
Number of potential warehouses:  26
Number of total warehouses:  44
Obs.: Each plant is also an existing warehouse


## 2. Parameters

In [3]:
# read state demands from Figure 1
state_demand = {
    # West region:
    "WA": 32437,  # Washington
    "OR": 31365,  # Oregon
    "CA": 135_116,  # California
    "NV": 16755,  # Nevada
    "AZ": 9063,  # Arizona
    "ID": 7153,  # Idaho
    # Northwest region:
    "UT": 9001,  # Utah
    "MT": 4140,  # Montana
    "ND": 5703,  # North Dakota
    "WY": 1004,  # Wyoming
    "CO": 11147,  # Colorado
    "SD": 1049,  # South Dakota
    "NE": 7347,  # Nebraska
    "KS": 6961,  # Kansas
    "MN": 5633,  # Minnesota
    "IA": 32175,  # Iowa
    "MO": 41680,  # Missouri
    # Southwest region:
    "NM": 3536,  # New Mexico
    "TX": 80438,  # Texas
    "OK": 13517,  # Oklahoma
    "AR": 4910,  # Arkansas
    "LA": 15011,  # Louisiana
    # Midwest region:
    "WI": 37448,  # Wisconsin
    "IL": 72839,  # Illinois
    "MI": 105_181,  # Michigan
    "IN": 43994,  # Indiana
    "KY": 3870,  # Kentucky
    "OH": 155_123,  # Ohio
    # Northeast region:
    "ME": 15829,  # Maine
    "NH": 4546,  # New Hampshire
    "RI": 17000,  # Rhode Island
    "NJ": 21154,  # New Jersey
    "NY": 160_917,  # New York
    "PA": 65108,  # Pennsylvania
    "CT": 26_187,  # Connecticut
    "MA": 37087,  # Massachusetts
    "VA": 17667,  # Virginia
    "WV": 9168,  # West Virginia
    "MD": 19284,  # Maryland
    "VT": 2928,  # Vermont
    "DE": 3044,  # Delaware
    # Southeast region:
    "TN": 42479,  # Tennessee
    "MS": 15_205,  # Mississippi
    "AL": 15835,  # Alabama
    "GA": 29559,  # Georgia
    "FL": 46405,  # Florida
    "SC": 5680,  # South Carolina
    "NC": 28_348,  # North Carolina
}
S = state_demand  # Sales

print("Total states number: ", len(S))
print("Total demand: ", sum(S.values()))
print("Each state represents a demand center")

# TODO: falta alguém ler e verificar os dados (o input foi feito de forma manual)

# NOTE: Valor total deve ser 1_477_026, então precisa ajustar NY (que nao tem na foto) pra bater o valor final.

Total states number:  48
Total demand:  1477026
Each state represents a demand center


In [4]:
demand_centers = [f"{s}" for s in state_demand.keys()]

pprint(demand_centers[:15])

print(
    """
    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    """
)

['WA',
 'OR',
 'CA',
 'NV',
 'AZ',
 'ID',
 'UT',
 'MT',
 'ND',
 'WY',
 'CO',
 'SD',
 'NE',
 'KS',
 'MN']

    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    


In [5]:
# 2.2 Plant current capacities C[p]
C = {
    "Covington": 620_000,
    "NewYork": 430_000,
    "Arlington": 300_000,
    "LongBeach": 280_000,
}

# Plant stocking capacities C'[p]
C_prime = {
    "Covington": 450_000,
    "NewYork": 380_000,
    "Arlington": 140_000,
    "LongBeach": 180_000,
}

In [6]:
# 2.3 Unit production cost v[p] (variable production cost)
rho = {
    "Covington": 21.0,
    "NewYork": 19.9,
    "Arlington": 21.6,
    "LongBeach": 21.1,
}

In [7]:
# 2.4 Distance matrices (in miles)
#     d_pw[p][w] = distance plant→warehouse
#     d_wj[w][j] = distance warehouse→demand center
# For now: mock with Euclidean or random coords or constants
d_pw = {p: {w: 300.0 for w in warehouses} for p in plants}
d_wj = {w: {j: 100.0 for j in demand_centers} for w in warehouses}
# TODO: compute real distances via geopy or similar

# TODO: ler do arquivo excel diretamente

In [8]:
# 2.5 Inbound cost c_in[p][w] = 0.92 + 0.0034*d_pw
c_in = {p: {w: 0.92 + 0.0034 * d_pw[p][w] for w in warehouses} for p in plants}

c_in

{'Covington': {'Atlanta': 1.94,
  'Boston': 1.94,
  'Buffalo': 1.94,
  'Chicago': 1.94,
  'Cleveland': 1.94,
  'Davenport': 1.94,
  'Detroit': 1.94,
  'GrandRapids': 1.94,
  'Greensboro': 1.94,
  'KansasCity': 1.94,
  'Baltimore': 1.94,
  'Memphis': 1.94,
  'Milwaukee': 1.94,
  'Orlando': 1.94,
  'Pittsburgh': 1.94,
  'Portland': 1.94,
  'WestSac': 1.94,
  'WestChester': 1.94,
  'Albuquerque': 1.94,
  'Billings': 1.94,
  'Denver': 1.94,
  'ElPaso': 1.94,
  'CampHill': 1.94,
  'Houston': 1.94,
  'LasVegas': 1.94,
  'Minneapolis': 1.94,
  'NewOrleans': 1.94,
  'Phoenix': 1.94,
  'Richmond': 1.94,
  'StLouis': 1.94,
  'SaltLakeCity': 1.94,
  'SanAntonio': 1.94,
  'Seattle': 1.94,
  'Spokane': 1.94,
  'SanFrancisco': 1.94,
  'Indianapolis': 1.94,
  'Louisville': 1.94,
  'Columbus': 1.94,
  'NewYork': 1.94,
  'Hartford': 1.94,
  'Miami': 1.94,
  'Mobile': 1.94,
  'Memphis2': 1.94,
  'Chicago2': 1.94},
 'NewYork': {'Atlanta': 1.94,
  'Boston': 1.94,
  'Buffalo': 1.94,
  'Chicago': 1.94,
  'C

In [9]:
# 2.6 Outbound cost c_out[w][j]:
local_rate = {w: 2.0 for w in warehouses}
# TODO: fill from Table 3’s "Local delivery rate" (pegar direto do excel)

c_out = {}
for w in warehouses:
    c_out[w] = {}
    for j in demand_centers:
        d = d_wj[w][j]
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            c_out[w][j] = local_rate[w]
        else:
            # else use 5.45 + 0.0037*d
            c_out[w][j] = 5.45 + 0.0037 * d

c_out

{'Atlanta': {'WA': 5.82,
  'OR': 5.82,
  'CA': 5.82,
  'NV': 5.82,
  'AZ': 5.82,
  'ID': 5.82,
  'UT': 5.82,
  'MT': 5.82,
  'ND': 5.82,
  'WY': 5.82,
  'CO': 5.82,
  'SD': 5.82,
  'NE': 5.82,
  'KS': 5.82,
  'MN': 5.82,
  'IA': 5.82,
  'MO': 5.82,
  'NM': 5.82,
  'TX': 5.82,
  'OK': 5.82,
  'AR': 5.82,
  'LA': 5.82,
  'WI': 5.82,
  'IL': 5.82,
  'MI': 5.82,
  'IN': 5.82,
  'KY': 5.82,
  'OH': 5.82,
  'ME': 5.82,
  'NH': 5.82,
  'RI': 5.82,
  'NJ': 5.82,
  'NY': 5.82,
  'PA': 5.82,
  'CT': 5.82,
  'MA': 5.82,
  'VA': 5.82,
  'WV': 5.82,
  'MD': 5.82,
  'VT': 5.82,
  'DE': 5.82,
  'TN': 5.82,
  'MS': 5.82,
  'AL': 5.82,
  'GA': 5.82,
  'FL': 5.82,
  'SC': 5.82,
  'NC': 5.82},
 'Boston': {'WA': 5.82,
  'OR': 5.82,
  'CA': 5.82,
  'NV': 5.82,
  'AZ': 5.82,
  'ID': 5.82,
  'UT': 5.82,
  'MT': 5.82,
  'ND': 5.82,
  'WY': 5.82,
  'CO': 5.82,
  'SD': 5.82,
  'NE': 5.82,
  'KS': 5.82,
  'MN': 5.82,
  'IA': 5.82,
  'MO': 5.82,
  'NM': 5.82,
  'TX': 5.82,
  'OK': 5.82,
  'AR': 5.82,
  'LA': 5.82

In [10]:
# TODO: precisa calcular isso daqui.
# custos de transporte da fabrica direto para o demand center.
c_out_prime = {}

for p in plants:
    c_out_prime[p] = {}
    for d in demand_centers:
        c_out_prime[p][d] = 1.0
        # TODO: pegar distâncias direto da planilha

In [11]:
# Quanto representa 1 cwt. em $? Dividimos as vendas totais ($) pela demand total (cwt)
Gamma = 160_000_000 / 1_477_026  # ($ / cwt)
Gamma

108.32578438023434

In [12]:
tau = {}

# TODO: precisa carregar esse tau direto do excel

for w in warehouses:
    tau[w] = 1.0

tau

{'Atlanta': 1.0,
 'Boston': 1.0,
 'Buffalo': 1.0,
 'Chicago': 1.0,
 'Cleveland': 1.0,
 'Davenport': 1.0,
 'Detroit': 1.0,
 'GrandRapids': 1.0,
 'Greensboro': 1.0,
 'KansasCity': 1.0,
 'Baltimore': 1.0,
 'Memphis': 1.0,
 'Milwaukee': 1.0,
 'Orlando': 1.0,
 'Pittsburgh': 1.0,
 'Portland': 1.0,
 'WestSac': 1.0,
 'WestChester': 1.0,
 'Albuquerque': 1.0,
 'Billings': 1.0,
 'Denver': 1.0,
 'ElPaso': 1.0,
 'CampHill': 1.0,
 'Houston': 1.0,
 'LasVegas': 1.0,
 'Minneapolis': 1.0,
 'NewOrleans': 1.0,
 'Phoenix': 1.0,
 'Richmond': 1.0,
 'StLouis': 1.0,
 'SaltLakeCity': 1.0,
 'SanAntonio': 1.0,
 'Seattle': 1.0,
 'Spokane': 1.0,
 'SanFrancisco': 1.0,
 'Indianapolis': 1.0,
 'Louisville': 1.0,
 'Columbus': 1.0,
 'NewYork': 1.0,
 'Hartford': 1.0,
 'Miami': 1.0,
 'Mobile': 1.0,
 'Memphis2': 1.0,
 'Chicago2': 1.0}

In [13]:
# 2.7 Warehouse operating cost per cwt: r_w[w]
#     = storage_rate + handling_rate   (from Table 3)
storage = {w: 0.1 for w in warehouses}
handling = {w: 0.7 for w in warehouses}
# TODO: replace storage[w] and handling[w] with Table 3 values


r_w = {w: storage[w] + handling[w] for w in warehouses}

In [14]:
# Big-M for linkage: no warehouse ships more than total demand
# M = sum(S.values())
# M

## 3. Defining the model

In [15]:
m = Model(name="UsemoreWarehousing")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-11


### Decision variables

In [16]:
Z = m.addVars(warehouses, vtype=GRB.BINARY, name="z")
X = m.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="x")
Y = m.addVars(warehouses, demand_centers, vtype=GRB.CONTINUOUS, name="y")
W = m.addVars(plants, demand_centers, vtype=GRB.CONTINUOUS, name="w")

## 4. Objective Function

In [17]:
m.setObjective(
    # inbound transport (from plants to warehouses)
    sum(c_in[i][j] * X[i, j] for i in plants for j in warehouses)
    # outbound transport (from warehouses to demand nodes)
    + sum(c_out[j][k] * Y[j, k] for j in warehouses for k in demand_centers)
    # transport cost from plants to demand centers
    + sum(c_out_prime[i][k] * W[i, k] for i in plants for k in demand_centers)
    # production cost
    + sum(
        rho[p] * (X[p, w] + W[p, d])
        for p in plants
        for w in warehouses
        for d in demand_centers
    )
    # storage cost in warehouses
    + sum(X[p, w] * Gamma * tau[w] for p in plants for w in warehouses),
    # TODO: add "custo de handling nos armazéns)"
    # TODO: add ""custo de processamento do pedido de estoque"
    # TODO: add "custo de processamento do pedido do cliente"
    sense=GRB.MINIMIZE,
)

## Constraints

In [18]:
# Demand satisfaction
m.addConstrs((Y.sum("*", d) == S[d] for d in demand_centers), "demand")

{'WA': <gurobi.Constr *Awaiting Model Update*>,
 'OR': <gurobi.Constr *Awaiting Model Update*>,
 'CA': <gurobi.Constr *Awaiting Model Update*>,
 'NV': <gurobi.Constr *Awaiting Model Update*>,
 'AZ': <gurobi.Constr *Awaiting Model Update*>,
 'ID': <gurobi.Constr *Awaiting Model Update*>,
 'UT': <gurobi.Constr *Awaiting Model Update*>,
 'MT': <gurobi.Constr *Awaiting Model Update*>,
 'ND': <gurobi.Constr *Awaiting Model Update*>,
 'WY': <gurobi.Constr *Awaiting Model Update*>,
 'CO': <gurobi.Constr *Awaiting Model Update*>,
 'SD': <gurobi.Constr *Awaiting Model Update*>,
 'NE': <gurobi.Constr *Awaiting Model Update*>,
 'KS': <gurobi.Constr *Awaiting Model Update*>,
 'MN': <gurobi.Constr *Awaiting Model Update*>,
 'IA': <gurobi.Constr *Awaiting Model Update*>,
 'MO': <gurobi.Constr *Awaiting Model Update*>,
 'NM': <gurobi.Constr *Awaiting Model Update*>,
 'TX': <gurobi.Constr *Awaiting Model Update*>,
 'OK': <gurobi.Constr *Awaiting Model Update*>,
 'AR': <gurobi.Constr *Awaiting Model Up

In [19]:
# Plant capacity
m.addConstrs((X.sum(p, "*") <= C[p] for p in plants), "plantCap")

{'Covington': <gurobi.Constr *Awaiting Model Update*>,
 'NewYork': <gurobi.Constr *Awaiting Model Update*>,
 'Arlington': <gurobi.Constr *Awaiting Model Update*>,
 'LongBeach': <gurobi.Constr *Awaiting Model Update*>}

In [ ]:
# TODO: restrição dos 10400

In [ ]:
# TODO: restrição do u_{pd}

In [ ]:
# Flow balance at each warehouse
# m.addConstrs(( x.sum('*',w) == y.sum(w,'*') for w in warehouses ), "flowBalance")


## Solver

In [20]:
m.params.TimeLimit = 300  # seconds

m.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 7730U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 52 rows, 2524 columns and 2288 nonzeros
Model fingerprint: 0x3a376cab
Variable types: 2480 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 6e+05]
Found heuristic solution: objective 8596291.3200
Presolve removed 52 rows and 2524 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 8.59629e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.596291320000e+06, best bound 8.5962913200

In [21]:
m.printStats()


Statistics for model UsemoreWarehousing:
  Linear constraint matrix    : 52 Constrs, 2524 Vars, 2288 NZs
  Variable types              : 2480 Continuous,
44 Integer (44 Binary)
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 5.82, 1147.07 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1004, 620000 ]


In [22]:
m.printQuality()


Solution quality statistics for model UsemoreWarehousing :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 0.00000000e+00
    Integrality : 0.00000000e+00


## Visualize results

In [23]:
print("\n--- Warehouse openings ---")
for w in warehouses:
    if Z[w].X > 0.1:
        print(f" Open warehouse at {w}")


--- Warehouse openings ---


In [24]:
print("\n--- Plant→Warehouse flows ---")
for p, w in X.keys():
    if X[p, w].X > 1e-6:
        print(f" {p} → {w}: {X[p, w].X:.0f} cwt")


--- Plant→Warehouse flows ---


In [25]:
print("\n--- Plant → Demand flows ---")
for w, j in W.keys():
    if W[w, j].X > 1e-6:
        print(f" {w:<13} → {j}: {W[w, j].X:.0f} cwt")


--- Plant → Demand flows ---


In [26]:
print("\n--- Warehouse→Demand flows ---")
for w, j in Y.keys():
    if Y[w, j].X > 1e-6:
        print(f" {w:<13} → {j}: {Y[w, j].X:.0f} cwt")


--- Warehouse→Demand flows ---
 Atlanta       → CO: 11147 cwt
 Buffalo       → WI: 37448 cwt
 Buffalo       → VA: 17667 cwt
 Chicago       → DE: 3044 cwt
 Chicago       → MS: 15205 cwt
 Cleveland     → IL: 72839 cwt
 Davenport     → KS: 6961 cwt
 Detroit       → WV: 9168 cwt
 GrandRapids   → IN: 43994 cwt
 GrandRapids   → MD: 19284 cwt
 KansasCity    → UT: 9001 cwt
 KansasCity    → ND: 5703 cwt
 KansasCity    → IA: 32175 cwt
 KansasCity    → RI: 17000 cwt
 Baltimore     → KY: 3870 cwt
 Baltimore     → ME: 15829 cwt
 Baltimore     → GA: 29559 cwt
 Orlando       → OK: 13517 cwt
 Orlando       → NH: 4546 cwt
 Pittsburgh    → OH: 155123 cwt
 Portland      → WA: 32437 cwt
 Albuquerque   → SD: 1049 cwt
 Denver        → OR: 31365 cwt
 Denver        → MN: 5633 cwt
 Denver        → NY: 160917 cwt
 Denver        → TN: 42479 cwt
 ElPaso        → NJ: 21154 cwt
 CampHill      → TX: 80438 cwt
 Houston       → CA: 135116 cwt
 Minneapolis   → NM: 3536 cwt
 NewOrleans    → ID: 7153 cwt
 NewOrleans    